In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy import signal
import os
import devicely
import zipfile
import re
import urllib.request
import openpyxl

# 1. Download nurses dataset
Download the dataset and extract it. The website is: https://datadryad.org/stash/dataset/doi:10.5061/dryad.5hqbzkh6f

In [15]:
%%time

# download if not already available
filename = 'doi_10.5061_dryad.5hqbzkh6f__v6.zip'
if not os.path.isfile(filename):
    print('downloading')
    urllib.request.urlretrieve('http://datadryad.org/api/v2/datasets/doi%253A10.5061%252Fdryad.5hqbzkh6f/download',
                               filename)

Wall time: 0 ns


In [4]:
%%time

# exctract if folder is not present
toFolder = 'doi_10.5061_dryad.5hqbzkh6f__v6'
if not os.path.isdir(toFolder):
    print('extracting')
    with zipfile.ZipFile(filename, 'r') as zfile:
        zfile.extractall(path=toFolder)

Wall time: 0 ns


# 2. Settings and extracting data

In [5]:
# supress warning that is not relevant for the code here
pd.options.mode.chained_assignment = None

In [6]:
# constant for time that need to be shifted for ACC data
cst = pd.Timedelta(hours=-6)

In [10]:
# function for extracting all data from the nurse dataset
def extract_nested_zip(zippedFile, toFolder):
    """ Extract a zip file including any nested zip files
        Delete the zip file(s) after extraction
    """
    with zipfile.ZipFile(zippedFile, 'r') as zfile:
        zfile.extractall(path=toFolder)
    os.remove(zippedFile)
    for root, dirs, files in os.walk(toFolder):
        for filename in files:
            if re.search(r'\.zip$', filename):
                zip_file = root+'/'+filename
                folder_name = zip_file[:-4]
                extract_nested_zip(zip_file, folder_name)

In [11]:
%%time

# unzip all folders
extract_folder = 'doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/'

if not os.path.isdir(extract_folder):
    os.makedirs(extract_folder)

file_to_extract = 'doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset.zip'

if os.path.isfile(file_to_extract):
    extract_nested_zip(file_to_extract, extract_folder)

Wall time: 1.01 ms


# 3. Extract all data from nurses dataset

In [12]:
# read survey excel file with labels
survey = pd.read_excel('doi_10.5061_dryad.5hqbzkh6f__v6/SurveyResults.xlsx',
                parse_dates=[['date', 'Start time'], ['date', 'End time']],
                dtype={'ID':str, 'Stress level':str})
survey = survey[['ID', 'date_Start time', 'date_End time', 'Stress level']]
survey = survey[survey['Stress level'] != 'na']
survey = survey[survey['Stress level'] != '1']
survey['Stress level'] = survey['Stress level'].replace({'2': '1'})
survey = survey.astype({'Stress level': int})
survey = survey.rename(columns={'ID':'subject', 'date_Start time':'dateStart', 'date_End time':'dateEnd', 'Stress level':'label'})

In [294]:
# function for getting a labeled dataframe from a subject's folder
# utilized the survey df
def get_df_from_folder(subject, folder):
    empatica_reader = devicely.EmpaticaReader(folder)
    empatica_reader.timeshift(cst)
    df = empatica_reader.data
    df = df[['ACC_X', 'ACC_Y', 'ACC_Z']]
    df = df.rename(columns={'ACC_X':'x', 'ACC_Y':'y', 'ACC_Z':'z'})
    df = df.dropna()
    df['label'] = np.nan
    df['subject'] = subject

    # get subject-df with all labels
    subject_labels_df = survey[survey['subject'] == subject]

    subject_df_list = []

    # iterate over each line of the subject-df and set labels in df
    for index, row in subject_labels_df.iterrows():
        start = row['dateStart']
        end = row['dateEnd']
        label = row['label']
        relevant_df = df.loc[start:end]
        relevant_df['label'] = label
        subject_df_list.append(relevant_df)
        
    subject_df = pd.concat(subject_df_list)
    return subject_df

In [295]:
%%time

# loop over all participants and all their folders and create the labled dfs
# concat all resulting dfs together into one
participants = [name for name in os.listdir('doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/')
                if os.path.isdir('doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/'+name)]

df_results_list = []

for p in participants:
    subfolders = [name for name in os.listdir('doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/'+p)
                if os.path.isdir('doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/'+p+'/'+name)]
    for sf in subfolders:
        folder = 'doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/'+p+'/'+sf
        df_curr = get_df_from_folder(p, folder)
        df_results_list.append(df_curr)

df_results = pd.concat(df_results_list)

Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594140175\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594149654\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594213322\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594220239\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594238057\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594298975\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594311202\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1594325201\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v

c:\Tools\anaconda3\envs\base2\lib\site-packages\devicely\empatica.py:187: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  return pd.read_csv(path, header=None,


Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595592772\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595600598\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595604812\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595608145\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595854651\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595857850\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595862148\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/15/15_1595862148\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f

c:\Tools\anaconda3\envs\base2\lib\site-packages\devicely\empatica.py:187: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  return pd.read_csv(path, header=None,


Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587297777\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587336033\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587338580\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587338580\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587569014\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587593082\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587593082\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/5C/5C_1587593875\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6

...

Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605101659\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605118411\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605118411\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605118579\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605186788\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605533971\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605625156\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1605881234\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__

c:\Tools\anaconda3\envs\base2\lib\site-packages\devicely\empatica.py:187: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  return pd.read_csv(path, header=None,


Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1607634439\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1607634439\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1607644800\tags.csv is empty.


c:\Tools\anaconda3\envs\base2\lib\site-packages\devicely\empatica.py:187: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  return pd.read_csv(path, header=None,


Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1608048804\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1608048804\tags.csv is empty.
Not reading signal because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1608049371\IBI.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/EG/EG_1608049371\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/F5/F5_1594141473\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/F5/F5_1594217466\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/F5/F5_1594218280\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset/F5/F5_1594223270\tags.csv is empty.
Not reading tags because the file doi_10.5061_dryad.5hqbzkh6f_

In [304]:
# drop index because we don't need it
df = df_results.reset_index()

In [306]:
df = df.drop(columns=['index'])

In [307]:
df

,x,y,z,label,subject
0,-52.0,15.0,-35.0,1,15
1,-52.0,15.0,-35.0,1,15
2,-52.0,15.0,-35.0,1,15
3,-56.0,15.0,-35.0,1,15
4,-56.0,14.0,-35.0,1,15
...,...,...,...,...,...
8543804,-28.0,-58.0,10.0,1,F5
8543805,-15.0,-60.0,21.0,1,F5
8543806,-10.0,-42.0,33.0,1,F5
8543807,-13.0,-31.0,35.0,1,F5


# 4. Store to Parquet file

In [308]:
# store as parquet

if not os.path.isdir('data-input'):
    os.makedirs('data-input')

df.to_parquet('data-input/dataset_nurses.parquet')